# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [ ]:
#from openai import OpenAI
#import os
#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv())

#OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [1]:
pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [2]:
from openai import OpenAI
import os

from google.colab import userdata

OPENAI_API_KEY = userdata.get('IronAPI')

In [20]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [34]:
#Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['Customers', 'Products', 'Orders'],
    'definition': [
        'Table containing information about customers',
        'Table containing information about products',
        'Table containing information about orders'
    ]
}

df = pd.DataFrame(data)
print(df)

       table                                    definition
0  Customers  Table containing information about customers
1   Products   Table containing information about products
2     Orders     Table containing information about orders


In [35]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [36]:
print(text_tables)

Customers: Table containing information about customers
Products: Table containing information about products
Orders: Table containing information about orders


In [37]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [41]:
#Creating the prompt, with the user questions and the tables definitions.
pqt1 = prompt_question_tables.format(tables=text_tables, question='Which tables are included in the dataset?')

In [42]:
print(return_OAI(pqt1))

```json
{
    "tables": ["Customers", "Products", "Orders"]
}
```


In [49]:
pqt3 = prompt_question_tables.format(question='Can you provide a list of tables along with their definitions?',
                                      tables=text_tables)

In [50]:
print(return_OAI(pqt3))

```json
{
    "Tables": [
        {
            "Table Name": "Customers",
            "Definition": "Table containing information about customers"
        },
        {
            "Table Name": "Products",
            "Definition": "Table containing information about products"
        },
        {
            "Table Name": "Orders",
            "Definition": "Table containing information about orders"
        }
    ]
}
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?

In [51]:
prompt1 = prompt_question_tables.format(
    question='What tables are available in the database?',
    tables=text_tables
)

In [52]:
print(return_OAI(prompt1))

```json
{
    "tables": ["Customers", "Products", "Orders"]
}
```


In [53]:
prompt2 = prompt_question_tables.format(
    question='Describe the tables in the database.',
    tables=text_tables
)

In [54]:
print(return_OAI(prompt2))

```json
{
    "tables": ["Customers", "Products", "Orders"]
}
```


In [56]:
prompt3 = prompt_question_tables.format(
    question='Can you list the tables along with their purposes?',
    tables=text_tables
)

In [57]:
print(return_OAI(prompt3))

```json
{
    "Tables": {
        "Customers": "Table containing information about customers",
        "Products": "Table containing information about products",
        "Orders": "Table containing information about orders"
    }
}
```


In [59]:
def generate_sql_query(table_name):
    return f"SELECT * FROM {table_name};"

In [60]:
def return_OAI(prompt):
    response = {
        "tables": ["Customers", "Products", "Orders"],
        "queries": {table: generate_sql_query(table) for table in ["Customers", "Products", "Orders"]}
    }
    return response

In [62]:
prompt = prompt_question_tables.format(
    question='Can you provide a list of tables along with their definitions?',
    tables=text_tables
)

In [63]:
def interactive_database_explorer():
    # Simulate initial interaction with AI to get tables and queries
    initial_response = return_OAI(prompt)
    tables = initial_response["tables"]
    queries = initial_response["queries"]

    print("Welcome to Interactive Database Explorer!")
    print("Available Tables:")
    for table in tables:
        print(f"- {table}")

    while True:
        print("\nWhat would you like to do?")
        print("1. Describe a table")
        print("2. Execute a SQL query")
        print("3. Exit")

        choice = input("Enter your choice (1/2/3): ")

        if choice == '1':
            table_name = input("Enter table name to describe: ")
            if table_name in tables:
                print(f"Table: {table_name}")
                print(f"Description: {data['definition'][tables.index(table_name)]}")
            else:
                print(f"Table '{table_name}' not found.")

        elif choice == '2':
            table_name = input("Enter table name to execute query: ")
            if table_name in tables:
                print(f"Executing query for '{table_name}':")
                print(queries[table_name])
            else:
                print(f"Table '{table_name}' not found.")

        elif choice == '3':
            print("Exiting...")
            break

        else:
            print("Invalid choice. Please enter 1, 2, or 3.")

# Execute the interactive database explorer
interactive_database_explorer()


Welcome to Interactive Database Explorer!
Available Tables:
- Customers
- Products
- Orders

What would you like to do?
1. Describe a table
2. Execute a SQL query
3. Exit
Enter your choice (1/2/3): 1
Enter table name to describe: Products
Table: Products
Description: Table containing information about products

What would you like to do?
1. Describe a table
2. Execute a SQL query
3. Exit
Enter your choice (1/2/3): 2
Enter table name to execute query: Orders
Executing query for 'Orders':
SELECT * FROM Orders;

What would you like to do?
1. Describe a table
2. Execute a SQL query
3. Exit


KeyboardInterrupt: Interrupted by user